In [1]:
# Core
import numpy as np
import pandas as pd

# Astronomy
import rebound

In [2]:
# Local imports
import kepler_sieve
from db_utils import sp2df

In [3]:
from asteroid_element import get_ast_elts, make_sim_asteroids
from orbital_element import anomaly_E2f, anomaly_f2E, anomaly_E2M, danby_guess, danby_iteration, anomaly_M2E_danby
from orbital_element import elt2pos, elt2vec
from orbital_element_test import make_test_elements, get_test_elements, angle_distance, report_test, test_E2f, test_E2M, test_M2E, test_all
from asteroid_data import ast_add_earth_elts, ast_add_sun_vectors

In [4]:
from planets_interp import get_earth_pos, get_earth_vectors, get_earth_elt, get_sun_vectors

In [5]:
# Get test orbital elements
elts = get_test_elements()

In [6]:
elts

,TimeID,AsteroidID,mjd,qx,qy,qz,vx,vy,vz,a,e,inc,Omega,omega,f,M
0,84960000,1,59000.0,2.200915,-1.931843,-0.467561,0.006340,0.007130,-0.000945,2.767657,0.077557,0.184806,1.401272,1.286860,-3.401546,2.839411
1,84960000,2,59000.0,0.662691,-2.706223,1.817727,0.008356,0.000283,-0.000904,2.773841,0.229972,0.607949,3.019851,-0.869132,2.742191,2.530303
2,84960000,3,59000.0,-2.901474,-1.192233,0.390143,0.001943,-0.008331,0.001812,2.668285,0.256936,0.226736,2.964468,-1.953614,2.535135,2.189260
3,84960000,4,59000.0,-0.240387,2.551044,-0.047390,-0.010162,-0.001270,0.001274,2.362014,0.088516,0.124650,1.811810,2.633265,-2.783176,-2.716995
4,84960000,5,59000.0,-2.063769,0.498221,0.084128,-0.003827,-0.012194,0.001120,2.574037,0.190913,0.093679,2.470881,-0.023590,0.461775,0.311477
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9988,84960000,9995,59000.0,1.929299,1.989779,0.047063,-0.006940,0.006451,0.000343,2.390249,0.160610,0.040347,0.363924,-2.826248,3.260392,-2.979609
9989,84960000,9996,59000.0,-0.850272,-2.870477,0.281759,0.009476,-0.000501,0.000683,2.796605,0.235174,0.133652,-2.629937,2.898821,-2.120904,-1.682728
9990,84960000,9997,59000.0,-2.235940,-1.662699,-0.104958,0.005379,-0.008184,-0.000224,2.545674,0.113059,0.042929,-0.427796,1.579063,-3.649961,2.514484
9991,84960000,9998,59000.0,1.949042,-0.287275,0.015234,0.001389,0.012640,0.000860,2.160961,0.093852,0.068282,-0.262023,0.578132,-0.465334,-0.386152


In [7]:
# Add vectors for the sun
elts_aug = elts.copy()
ast_add_earth_elts(elts_aug)
ast_add_sun_vectors(elts_aug)


In [8]:
elts_aug

,TimeID,AsteroidID,mjd,qx,qy,qz,vx,vy,vz,a,...,earth_Omega,earth_omega,earth_f,earth_M,sun_qx,sun_qy,sun_qz,sun_vx,sun_vy,sun_vz
0,84960000,1,59000.0,2.200915,-1.931843,-0.467561,0.006340,0.007130,-0.000945,2.767657,...,3.021905,-1.251463,2.588308,2.570585,-0.00504,0.007027,0.000058,-0.000008,-0.000004,2.328685e-07
1,84960000,2,59000.0,0.662691,-2.706223,1.817727,0.008356,0.000283,-0.000904,2.773841,...,3.021905,-1.251463,2.588308,2.570585,-0.00504,0.007027,0.000058,-0.000008,-0.000004,2.328685e-07
2,84960000,3,59000.0,-2.901474,-1.192233,0.390143,0.001943,-0.008331,0.001812,2.668285,...,3.021905,-1.251463,2.588308,2.570585,-0.00504,0.007027,0.000058,-0.000008,-0.000004,2.328685e-07
3,84960000,4,59000.0,-0.240387,2.551044,-0.047390,-0.010162,-0.001270,0.001274,2.362014,...,3.021905,-1.251463,2.588308,2.570585,-0.00504,0.007027,0.000058,-0.000008,-0.000004,2.328685e-07
4,84960000,5,59000.0,-2.063769,0.498221,0.084128,-0.003827,-0.012194,0.001120,2.574037,...,3.021905,-1.251463,2.588308,2.570585,-0.00504,0.007027,0.000058,-0.000008,-0.000004,2.328685e-07
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9988,84960000,9995,59000.0,1.929299,1.989779,0.047063,-0.006940,0.006451,0.000343,2.390249,...,3.021905,-1.251463,2.588308,2.570585,-0.00504,0.007027,0.000058,-0.000008,-0.000004,2.328685e-07
9989,84960000,9996,59000.0,-0.850272,-2.870477,0.281759,0.009476,-0.000501,0.000683,2.796605,...,3.021905,-1.251463,2.588308,2.570585,-0.00504,0.007027,0.000058,-0.000008,-0.000004,2.328685e-07
9990,84960000,9997,59000.0,-2.235940,-1.662699,-0.104958,0.005379,-0.008184,-0.000224,2.545674,...,3.021905,-1.251463,2.588308,2.570585,-0.00504,0.007027,0.000058,-0.000008,-0.000004,2.328685e-07
9991,84960000,9998,59000.0,1.949042,-0.287275,0.015234,0.001389,0.012640,0.000860,2.160961,...,3.021905,-1.251463,2.588308,2.570585,-0.00504,0.007027,0.000058,-0.000008,-0.000004,2.328685e-07


In [10]:
key_cols = elts.columns[0:3]
elts[key_cols]

,TimeID,AsteroidID,mjd
0,84960000,1,59000.0
1,84960000,2,59000.0
2,84960000,3,59000.0
3,84960000,4,59000.0
4,84960000,5,59000.0
...,...,...,...
9988,84960000,9995,59000.0
9989,84960000,9996,59000.0
9990,84960000,9997,59000.0
9991,84960000,9998,59000.0


In [7]:
# test_all()

In [7]:
# Unpack orbital elements
a = elts.a.values
e = elts.e.values
inc = elts.inc.values
Omega = elts.Omega.values
omega = elts.omega.values
f = elts.f.values
M = elts.M.values

In [9]:
# The state vectors according to the integration
cols_q = ['qx', 'qy', 'qz']
cols_v = ['vx', 'vy', 'vz']
q: np.array = elts[cols_q].values
v: np.array = elts[cols_v].values


In [24]:
# State vectors from elements
q_hel, v_hel = elt2vec(a=a, e=e, inc=inc, Omega=Omega, omega=omega, f=f)

In [25]:
# State vectors for sun
ts = elts.mjd.values
q_sun, v_sun = get_sun_vectors(ts)

In [26]:
# Barycentric state vectors from elements
q2 = q_hel + q_sun
v2 = v_hel + v_sun

In [27]:
# difference in vectors
dq = q2 - q
dv = v2 - v

# error in vectors
err_q: np.array = np.sqrt(np.sum(np.square(dq), axis=-1))
err_v: np.array = np.sqrt(np.sum(np.square(dv), axis=-1))

In [29]:
np.mean(err_q)

4.233503582580232e-15

In [30]:
err_v

array([3.49542859, 3.06629608, 3.18467013, ..., 3.56878981, 4.64392038,
       3.92160801])

In [31]:
v[0]

array([ 0.00634042,  0.00713024, -0.00094455])

In [32]:
v2[0]

array([ 2.31879481,  2.60561919, -0.34508249])

In [33]:
np.linalg.norm(v[0])

0.009588193859724448

In [35]:
np.linalg.norm(v2[0])

3.5050167831417713

In [36]:
np.linalg.norm(v2[0])  / np.linalg.norm(v[0])

365.5554773318383

In [37]:
from rebound_sim import make_sim_planets

In [38]:
sim = make_sim_planets(epoch=59000)

In [40]:
print(f'{sim.G:20.18e}')

2.959122082855910945e-04
